In [1]:
import os
# librerías de google cloud
from google.cloud import dialogflow_v2beta1 as dialogflow
from google.api_core.exceptions import InvalidArgument
import cv2

In [2]:
# conecto con el chatbot de Dialogflow
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'DialogdlowKey.json'

DIALOGFLOW_PROJECT_ID = 'traductorbot-whkt'
DIALOGFLOW_LANGUAGE_CODE = '[en]'
SESSION_ID = 'me'


In [3]:
print('Antes de comenzar, asegúrese de que el texto que desea traducir a Lenguaje de Signos en Español sea en castellano')
print('Intentaremos proporcionar la mejor traducción de su texto')
print('')
print('¿Qué texto desea traducir?')
frase = input()

Antes de comenzar, asegúrese de que el texto que desea traducir a Lenguaje de Signos en Español sea en castellano
Intentaremos proporcionar la mejor traducción de su texto

¿Qué texto desea traducir?
Es UnA MuJEr Fuerte y CORRE muy RÁPido


In [4]:
frase=frase.lower()
frase

'es una mujer fuerte y corre muy rápido'

In [5]:
# función para eliminar las tildes y diéresis
def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        ("ä", "a"),
        ("ë", "e"),
        ("ï", "i"),
        ("ö", "o"),
        ("ü", "u"),     
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

In [6]:
frase=normalize(frase)
frase

'es una mujer fuerte y corre muy rapido'

In [7]:
length = len(frase.split())
length

8

In [8]:
frase=frase.split()
frase

['es', 'una', 'mujer', 'fuerte', 'y', 'corre', 'muy', 'rapido']

In [9]:
for i in range(0, length):   
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(DIALOGFLOW_PROJECT_ID, SESSION_ID)
    text_input = dialogflow.types.TextInput(text=frase[i], language_code=DIALOGFLOW_LANGUAGE_CODE)
    query_input = dialogflow.types.QueryInput(text=text_input)
    try:
        response = session_client.detect_intent(session=session, query_input=query_input)
    except InvalidArgument:
        raise
    # Imprimo la palabra que se va a traducir
    print("Query text:", response.query_result.query_text)
    # Imprimo la apalbra/frase de donde el chatbot extrae el vídeo
    # Elimino los primeros 28 caracteres ya que las primeras 28 letras de los intents son
    # Knowledge.Database.Palabras.
    print("Detected intent:", response.query_result.intent.display_name[28:])
    # Imprimo la confianza de que la palabra sea la correcta o similar
    print("Detected intent confidence:", response.query_result.intent_detection_confidence)
    # Imprimo el identificador del vídeo
    print("Fulfillment text:", response.query_result.fulfillment_text)
    
    # Código para reproducir los vídeos
    capture = cv2.VideoCapture('./Videos/'+response.query_result.fulfillment_text+'.mp4')

    while (capture.isOpened()):
        ret, frame = capture.read()
        if (ret == True):
            cv2.imshow(response.query_result.intent.display_name, frame)
            if (cv2.waitKey(30) == ord('s')):
                break
        else:
            break

    capture.release()
    cv2.destroyAllWindows()

Query text: es
Detected intent: es
Detected intent confidence: 1.0
Fulfillment text: 15566
Query text: una
Detected intent: una
Detected intent confidence: 1.0
Fulfillment text: 13543
Query text: mujer
Detected intent: mujer de color
Detected intent confidence: 0.32802149653434753
Fulfillment text: 19011
Query text: fuerte
Detected intent: fuerte
Detected intent confidence: 1.0
Fulfillment text: 15662
Query text: y
Detected intent: y
Detected intent confidence: 1.0
Fulfillment text: 13516
Query text: corre
Detected intent: 
Detected intent confidence: 1.0
Fulfillment text: Perdón, no disponemos de ese vídeo
Query text: muy
Detected intent: 
Detected intent confidence: 1.0
Fulfillment text: Perdón, no disponemos de ese vídeo
Query text: rapido
Detected intent: rapido
Detected intent confidence: 1.0
Fulfillment text: 18046
